In [1]:
%matplotlib inline
import numpy as np
import torch
from torch.utils.serialization import load_lua
import os
import scipy.io as sio
import cv2
import math
from matplotlib import pyplot as plt
from torch.utils.data.dataset import Dataset

from sklearn.decomposition import PCA

In [2]:
TRAIN_DATA_DIR = '/Data/nruiz9/data/facial_landmarks/300W_LP/'

In [4]:
shape_params_list = []
names = []

with open(os.path.join(TRAIN_DATA_DIR, 'filename_list_filtered.txt')) as f:
    for idx, line in enumerate(f):
        original_line = line
        line = line.strip('\n')
        mat_path = os.path.join(TRAIN_DATA_DIR, line + '.mat')
        mat = sio.loadmat(mat_path)

        shape_params_list.append(np.array(mat['Shape_Para'][:,0]))
        names.append(line)

In [5]:
X = [sp for sp in shape_params_list]
print len(X)

122415


In [6]:
pca = PCA(n_components=10)
pca.fit(X)
print(pca.explained_variance_ratio_)
print sum(pca.explained_variance_ratio_)

[ 0.4954199   0.13912562  0.11082269  0.07658631  0.04858431  0.02813001
  0.01758898  0.01631346  0.01002331  0.00814171]
0.950736293253


In [7]:
new_X = pca.transform(X)
print len(new_X)

122415


In [8]:
print np.amin(new_X, 0)
print np.amax(new_X, 0)
print np.median(new_X, 0)

print np.abs(np.amax(new_X, 0) - np.amin(new_X, 0))
print np.ptp(new_X, 0)
print new_X.shape

[-2556323.53165033 -1140679.77655202 -1371614.65446089 -1119583.33472875
  -754535.15912456  -821857.44375021  -534676.82835068  -499987.22801558
  -426309.71017172  -446477.70288007]
[ 5002830.39467843  1820495.74291976  1441834.85901925  1429397.04589404
  1223356.93869817   924078.41303862   760271.63357968   805551.96259901
   466004.54029864   545186.01838144]
[-176340.23999369     440.18163591   -2284.73154146   -6407.94592961
  -11806.29047248   -2078.74081526   -3059.95275478   -5356.3932487
   -3081.66281823    2027.99147229]
[ 7559153.92632876  2961175.51947178  2813449.51348013  2548980.38062279
  1977892.09782273  1745935.85678882  1294948.46193037  1305539.19061459
   892314.25047037   991663.72126151]
[ 7559153.92632876  2961175.51947178  2813449.51348013  2548980.38062279
  1977892.09782273  1745935.85678882  1294948.46193037  1305539.19061459
   892314.25047037   991663.72126151]
(122415, 10)


In [9]:
maxs = np.amax(new_X, 0)
mins = np.amin(new_X, 0)
dividers = 60
step_sizes = np.ptp(new_X, 0) / (dividers - 2)

bins = []
for idx in xrange(new_X.shape[1]):
    rng = range(int(mins[idx]), int(maxs[idx]) + 1, int(step_sizes[idx]))
    bins.append(np.digitize(new_X[:,idx], rng))
    
bins = np.array(bins)
print bins.shape
print np.amin(bins, 1)
print np.amax(bins, 1)

(10, 122415)
[0 0 0 0 0 0 0 0 0 0]
[59 59 59 59 59 59 59 59 59 59]


In [10]:
print names[0], names[1]

AFW_Flip/AFW_5083671561_5_5 AFW_Flip/AFW_1648807314_2_0


In [12]:
# Save the new PCA binned representation
idx = 0
for name in names:
    pose_path = os.path.join(TRAIN_DATA_DIR, name + '_shape.npy')
    np.save(pose_path, bins[:,idx])
    idx += 1